In [1]:
import pandas as pd
import rstr
from faker import Faker
from faker_e164.providers import E164Provider
fake = Faker()
fake.add_provider(E164Provider)
from datetime import datetime, date
import random
import hashlib
import json

In [2]:
def hash(name, size=10):
    if name is None: return False
    return hashlib.md5(name.encode('utf-8')).hexdigest()[:size]

In [3]:
hash(fake.name())

'be2f01d609'

In [4]:
def genId(prefix="", size=8):
    return prefix+''.join( rstr.xeger(r'[A-Z0-9]') for i in range(size))

In [5]:
def initBootstrap():
    return  {
    "_config": {},
    "couchello_db": {
      "_design": {},
      "_local": {},
      "_security": {
        "members": {
          "roles": [],
          "names": []
        },
        "admins": {
          "roles": [],
          "names": []
        }
        }
    },
    "_users": {}
}

In [6]:
fake.ascii_email()

'destiny59@deleon.com'

In [14]:
def createUsers(data):
    u = {}
    s = data["couchello_db"]["_security"]
    g = {"Admin": "admins", "Member": "members"}
    for i in range(10):
        name = fake.name()
        user = {}
        _id = "org.couchdb.user:"+hash(name)
        user["_id"] = _id
        user["type"] = "user"
        user["password"] = hash(name)
        user["roles"] = []
        user["name"] = hash(name)
        role = random.choice(["Admin", "Member"])
        s[g[role]]["names"].append(hash(name))
        u[_id] = user
    data["couchello_db"]["_security"] = s
    data["_users"] = u
    return data

In [11]:
def createOrganizations(r, size=1):
    r['organizations']={}
    for _ in range(size):
        n = fake.company() + fake.company_suffix()
        _id = hash(n)
        db = "org_"+_id+"_db"
        members = []
        [members.append(r["_users"][u]['name']) and  r["_users"][u]['roles'].append(db) for u in r["_users"]]
        r['organizations']['id'] = {
            "_id":_id, 
            "type":"organization", 
            "name": n, "db":db, "members":members
        }
    return r

In [12]:
def createBootstrap():
    r = initBootstrap()
    r = createUsers(r)
    r = createOrganizations(r)
    return r

In [15]:
createBootstrap()

KeyError: 'role'

In [ ]:
with open('bootstrap.header.txt', 'r') as infile:
    with open('bootstrap.js', 'w') as outfile:
        outfile.write(infile.read()+json.dumps(createBootstrap(), indent=4)+";")    

In [ ]:
with open('bootstrap.js', 'r') as file:
    print(file.read())